In [42]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report

train_path = './Data/train.csv'
test_path = './Data/test.csv'
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [43]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

X_train = train_data.drop(columns=['label']).values
Y_train = train_data['label'].values
X_test = test_data.values

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
stratified_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=43)
for fold, indi in enumerate(stratified_fold.split(X_train, Y_train)):
  X_train_, Y_train_ = X_train[indi[0]], Y_train[indi[0]]
  X_test_, Y_test_ = X_train[indi[1]], Y_train[indi[1]]

  # a k of 1 or 3 was found to have the best accuracy for this dataset
  kNN = KNeighborsClassifier(n_neighbors=3)
  kNN.fit(X_train_, Y_train_)
  pred = kNN.predict(X_test_)
  print(f"Classification report for Fold {fold + 1}:")
  print(classification_report(Y_test_, pred, digits=3), end="\n\n")
    
  print(f"Confusion Matrix for Fold {fold + 1}:")
  print(confusion_matrix(Y_test_, pred), end="\n\n")
  
  del X_train_
  del X_test_
  del Y_train_
  del Y_test_

Classification report for Fold 1:
              precision    recall  f1-score   support

           0      0.965     0.989     0.977       660
           1      0.949     0.996     0.972       752
           2      0.985     0.949     0.966       680
           3      0.957     0.948     0.953       699
           4      0.973     0.950     0.961       656
           5      0.942     0.956     0.949       615
           6      0.972     0.986     0.979       666
           7      0.960     0.958     0.959       709
           8      0.993     0.924     0.957       656
           9      0.941     0.972     0.956       676

    accuracy                          0.963      6769
   macro avg      0.964     0.963     0.963      6769
weighted avg      0.964     0.963     0.963      6769


Confusion Matrix for Fold 1:
[[653   0   0   0   0   1   6   0   0   0]
 [  0 749   2   0   0   0   0   1   0   0]
 [  6  12 645   0   0   1   1  14   1   0]
 [  3   1   6 663   0  16   1   3   2   4]
 [  2